In [53]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [54]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    
    node=CustomNode(suffixes[0],state_number,suffixes[1],True,root_node)
    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode('^s',state_number,'+PL',True,node)
        state_number+=1
        CustomNode("^s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+V+3D',True,node)
        state_number+=1
        CustomNode("ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_verb(stem,state_number,root):
    verbs=stems=stem.split('/')
    root_stem=verbs[0]
    for i in range(len(root_stem)):
        state_number+=1
        root=CustomNode(stem[i],state_number,'<eps>',False,root)
        
    state_number+=1
    n=CustomNode(verbs[0],state_number,verbs[0]+'+V',True,root)
    
    if '&' in verbs[1]: #'were&was'
        past_verbs=verbs[1].split('&')
        state_number+=1
        n=CustomNode(past_verbs[0],state_number,past_verbs[0]+'+V+PT',True,root)
        state_number+=1
        n=CustomNode(past_verbs[1],state_number,past_verbs[1]+'+V+3D+PT',True,root)
    else:
        state_number+=1
        n=CustomNode(verbs[1],state_number,verbs[1]+'+V+PT',True,root)
    state_number+=1
    n=CustomNode(verbs[2],state_number,verbs[2]+'+V+PP',True,root)
            
    return (root,state_number)

In [55]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Verbs.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_verb(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            state_number+=1
            CustomNode("<eps>",state_number,'+V',True,root)
            state_number+=1
            CustomNode("^s",state_number,'+3D',True,root)
            state_number+=1
            CustomNode("ing",state_number,'+ING',True,root)
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

In [56]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

<eps> 1parent: root
^s 2parent: root
ing 3parent: root
a 4parent: root
c 5parent: a
c 6parent: c
e 7parent: c
p 8parent: e
t 9parent: p
<eps> 10parent: t
^s 11parent: t
ing 12parent: t
k 13parent: c
n 14parent: k
o 15parent: n
w 16parent: o
l 17parent: w
e 18parent: l
d 19parent: e
g 20parent: d
e 21parent: g
<eps> 22parent: e
^s 23parent: e
ing 24parent: e
t 25parent: c
<eps> 26parent: t
^s 27parent: t
ing 28parent: t
d 29parent: a
m 30parent: d
i 31parent: m
r 32parent: i
e 33parent: r
<eps> 34parent: e
^s 35parent: e
ing 36parent: e
v 37parent: d
i 38parent: v
s 39parent: i
e 40parent: s
<eps> 41parent: e
^s 42parent: e
ing 43parent: e
i 44parent: a
m 45parent: i
<eps> 46parent: m
^s 47parent: m
ing 48parent: m
l 49parent: a
l 50parent: l
o 51parent: l
w 52parent: o
<eps> 53parent: w
^s 54parent: w
ing 55parent: w
n 56parent: a
s 57parent: n
w 58parent: s
e 59parent: w
r 60parent: e
<eps> 61parent: r
^s 62parent: r
ing 63parent: r
t 64parent: a
t 65parent: t
a 66parent: t
c 67parent

In [57]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	+V	<eps>
4	0	<eps>	#
0	5	+3D	^s
5	0	<eps>	#
0	6	+ING	ing
6	0	<eps>	#
0	7	a	a
7	8	c	c
8	9	c	c
9	10	e	e
10	11	p	p
11	12	t	t
12	13	+V	<eps>
13	0	<eps>	#
12	14	+3D	^s
14	0	<eps>	#
12	15	+ING	ing
15	0	<eps>	#
8	16	k	k
16	17	n	n
17	18	o	o
18	19	w	w
19	20	l	l
20	21	e	e
21	22	d	d
22	23	g	g
23	24	e	e
24	25	+V	<eps>
25	0	<eps>	#
24	26	+3D	^s
26	0	<eps>	#
24	27	+ING	ing
27	0	<eps>	#
8	28	t	t
28	29	+V	<eps>
29	0	<eps>	#
28	30	+3D	^s
30	0	<eps>	#
28	31	+ING	ing
31	0	<eps>	#
7	32	d	d
32	33	m	m
33	34	i	i
34	35	r	r
35	36	e	e
36	37	+V	<eps>
37	0	<eps>	#
36	38	+3D	^s
38	0	<eps>	#
36	39	+ING	ing
39	0	<eps>	#
32	40	v	v
40	41	i	i
41	42	s	s
42	43	e	e
43	44	+V	<eps>
44	0	<eps>	#
43	45	+3D	^s
45	0	<eps>	#
43	46	+ING	ing
46	0	<eps>	#
7	47	i	i
47	48	m	m
48	49	+V	<eps>
49	0	<eps>	#
48	50	+3D	^s
50	0	<eps>	#
48	51	+ING	ing
51	0	<eps>	#
7	52	l	l
52	53	l	l
53	54	o	o
54	55	w	w
55	56	+V	<eps>
56	0	<eps>	#
55	57	+3D	^s
57	0	<

In [30]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)